In [ ]:
from openpi.shared import download
checkpoint_dir = download.maybe_download("s3://openpi-assets/checkpoints/pi0_fast_base")

  0%|          | 0.00/10.1G [00:00<?, ?iB/s]

: 

In [1]:
import dataclasses

import jax

from openpi.models import model as _model
from openpi.policies import droid_policy
from openpi.policies import policy_config as _policy_config
from openpi.shared import download
from openpi.training import config as _config
from openpi.training import data_loader as _data_loader

# Policy inference

The following example shows how to create a policy from a checkpoint and run inference on a dummy example.

In [5]:
config = _config.get_config("pi0_fast_droid")
checkpoint_dir = download.maybe_download("s3://openpi-assets/checkpoints/pi0_fast_droid")

# Create a trained policy.
policy = _policy_config.create_trained_policy(config, checkpoint_dir)

# Run inference on a dummy example. This example corresponds to observations produced by the DROID runtime.
example = droid_policy.make_droid_example()
result = policy.infer(example)

# Delete the policy to free up memory.
del policy

print("example:", example)
print("result:", result)
print("Actions shape:", result["actions"].shape)

Some kwargs in processor config are unused and will not have any effect: vocab_size, time_horizon, action_dim, min_token, scale. 
Some kwargs in processor config are unused and will not have any effect: vocab_size, time_horizon, action_dim, min_token, scale. 


example: {'observation/exterior_image_1_left': array([[[ 43,  20, 203],
        [251, 190,  72],
        [ 62, 214, 241],
        ...,
        [150, 183, 173],
        [152, 100,  24],
        [ 15,   6, 249]],

       [[193,  61, 222],
        [ 96, 253, 174],
        [ 54, 236, 164],
        ...,
        [ 54,  92, 168],
        [  3, 152, 129],
        [129,  64,  39]],

       [[199, 128, 254],
        [ 73, 121, 229],
        [174, 128,  62],
        ...,
        [254, 140, 153],
        [197, 222, 229],
        [  9, 149, 227]],

       ...,

       [[250, 147,  52],
        [228, 153, 186],
        [254,  84, 123],
        ...,
        [ 95, 191, 138],
        [ 36, 134, 240],
        [  3, 201,  63]],

       [[131, 113,  48],
        [ 61, 210, 128],
        [166, 147, 109],
        ...,
        [ 51, 225, 158],
        [246, 153,  42],
        [117, 210,  53]],

       [[251,   6, 244],
        [167, 211,   3],
        [127,  63,  42],
        ...,
        [ 33, 179, 227],
  

# Working with a live model


The following example shows how to create a live model from a checkpoint and compute training loss. First, we are going to demonstrate how to do it with fake data.


In [6]:
config = _config.get_config("pi0_aloha_sim")

checkpoint_dir = download.maybe_download("s3://openpi-assets/checkpoints/pi0_aloha_sim")
key = jax.random.key(0)

# Create a model from the checkpoint.
model = config.model.load(_model.restore_params(checkpoint_dir / "params"))

# We can create fake observations and actions to test the model.
obs, act = config.model.fake_obs(), config.model.fake_act()

# Sample actions from the model.
loss = model.compute_loss(key, obs, act)
print("Loss shape:", loss.shape)

Loss shape: (1, 50)


Now, we are going to create a data loader and use a real batch of training data to compute the loss.

In [10]:
# Reduce the batch size to reduce memory usage.
config = dataclasses.replace(config, batch_size=2)

# Load a single batch of data. This is the same data that will be used during training.
# NOTE: In order to make this example self-contained, we are skipping the normalization step
# since it requires the normalization statistics to be generated using `compute_norm_stats`.
loader = _data_loader.create_data_loader(config, num_batches=1, skip_norm_stats=True)
obs, act = next(iter(loader))

# Sample actions from the model.
loss = model.compute_loss(key, obs, act)

# Delete the model to free up memory.
del model

print("Loss shape:", loss.shape)

ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /api/datasets/lerobot/aloha_sim_transfer_cube_human/refs (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x1510161a0690>: Failed to resolve \'huggingface.co\' ([Errno -2] Name or service not known)"))'), '(Request ID: 78862be2-8a60-4707-8dbd-0ec20468aaa6)')